In [1]:
cd RNN-Time-series-Anomaly-Detection/

/home/shviam/Documents/Upwork/bonhwa/RNN-Time-series-Anomaly-Detection


In [24]:
import numpy as np
import pandas as pd
from multiprocessing.pool import ThreadPool
import csv
import matplotlib.pyplot as plt
import argparse
import time
import torch
import torch.nn as nn
import preprocess_data
from model import model
from torch import optim
from pathlib import Path
import pickle
from anomalyDetector import fit_norm_distribution_param
%matplotlib inline

In [3]:
fgs = pd.read_csv('../fgs.csv')

In [4]:
fgs.head()

,,,,4963594775,1,2
proxy.9.5698,traffic counter.7.5664,0.971442,82.243099,1.794354,9.341767,100.0
5402488769,5,1.000000,NaN,NaN,NaN,NaN
traffic counter.7.7413,proxy.9.11244,4.060322,119.086081,16.548350,8.474562,150.0
proxy.9.11244,proxy.9.2362,0.268318,23.865256,11.573841,9.248256,150.0
proxy.9.2362,traffic counter.7.9524,2.961259,80.083566,18.280812,4.783467,150.0


In [5]:
vns = pd.read_csv('../vnfs.csv', header=None)

In [6]:
vnf_id = vns[vns.columns[0]]
vnf_type = vns[vns.columns[1]]
vnf_scheduling = vns[vns.columns[2]]
vnf_pm = vns[vns.columns[3]]
vnf_fg = vns[vns.columns[4]]
flavor_data = vns[vns.columns[5:8]]
vm_data = vns[vns.columns[8:11]]
usage_data = vns[vns.columns[11:15]]

In [7]:
output = pd.read_csv('../new_results_out.csv', sep =';')#header=None)
output.drop('Unnamed: 0',axis = 1, inplace=True)

In [8]:
output.head()

,vnf label,vnf min cpu,vnf min mem,vnf min sto,vnf vm cpu,vnf vm mem,vnf vm sto,vnf pm id,vnf cpu usage,vnf mem usage,...,EXTERNAL_TEMP,ROOM_TEMP,MTTF_IC,A_TC,Q_DIT,TPF,AIRFLOW,TAAF,DeltaT_de,QD
0,dpi.2.136144,0.035914,0.226393,0.001757,0.06250,0.12720,0.001930,257345416,71.88000,1.75780,...,15,23,2435.249666,0.996726,1.839678,14.999377,0.131858,0.872117,6.132259,6.132259
1,traffic counter.7.40368,0.063818,0.017674,0.000223,0.06189,0.03821,0.000386,3488796635,3.51600,6.34800,...,15,24,49.077683,0.859840,1.801941,14.999402,0.142946,0.768427,6.006469,6.006469
2,proxy.9.13456,0.018672,0.020912,0.000006,0.01250,0.01590,0.000404,4820294844,0.18044,0.13352,...,55,25,82.372536,0.911478,1.800100,54.999420,0.152569,0.757640,6.000332,6.000332
3,firewall.3.36933,0.140469,0.021706,0.000068,0.09375,0.03198,0.000070,3550430599,26.46000,6.44600,...,40,23,0.044220,0.005497,1.814606,39.999481,0.132889,0.807089,6.048686,6.048686
4,dpi.2.92594,0.015708,0.050654,0.000810,0.06250,0.08569,0.000926,4820072706,41.74000,14.42800,...,21,29,0.011200,0.001398,1.823040,20.999427,0.189031,0.790021,6.076802,6.076802


In [9]:
output.columns

Index(['vnf label', 'vnf min cpu', 'vnf min mem', 'vnf min sto', 'vnf vm cpu',
       'vnf vm mem', 'vnf vm sto', 'vnf pm id', 'vnf cpu usage',
       'vnf mem usage', 'vnf sto usage', 'vnf before', 'vnf after',
       'vnf label.1', 'vnf min cpu.1', 'vnf min mem.1', 'vnf min sto.1',
       'vnf vm cpu.1', 'vnf vm mem.1', 'vnf vm sto.1', 'vnf pm id.1',
       'vnf cpu usage.1', 'vnf mem usage.1', 'vnf sto usage.1', 'vnf before.1',
       'vnf after.1', 'sla', 'fg id', 'flow traffic', 'flow latency',
       'flow bnd usage', 'flow packet loss', 'min cpu affinity',
       'min mem affinity', 'min sto affinity', 'conflicts affinity',
       'real_affinity', 'static_affinity', 'predicted_affinity',
       'prediction_time', 'TF', 'MTT', 'MTT_upper', 'MTT_lower', 'MTTF_R',
       'MTTF_EM', 'MTTF_C', 'MTTF_TDDB', 'MTTF_SM', 'MTTFF_TC', 'A', 'AEM',
       'AC', 'ATDDB', 'ASM', 'ATC', 'TAA', 'QRED', 'QR', 'PUE', 'DCie', 'cost',
       'TIMESTAMP', 'EXTERNAL_TEMP', 'ROOM_TEMP', 'MTTF_IC', 'A_T

In [11]:
output['vnf cpu usage'].shape

(195379,)

In [12]:
output = output[:12322]
output = output['vnf cpu usage']

In [22]:
dataset = np.stack((output, vnf_id, np.zeros(len(vnf_id))), axis = 1).tolist()

In [23]:
dataset[0]

[71.88, 126.0, 0.0]

In [27]:
with open('train_dataset','wb') as pkl:
    pickle.dump(dataset[:12000], pkl)
    
with open('test_dataset','wb') as pkl:
    pickle.dump(dataset[12000:], pkl)

In [22]:
train_dataset = vns[:10000]
trainy = output[:10000]

test_dataset = vns[10000:]
testy = output[10000:]

[[71.88, 126.0, 0.0],
 [3.516, 128.0, 0.0],
 [0.18044, 130.0, 0.0],
 [26.46, 135.0, 0.0],
 [41.74, 273.0, 0.0],
 [36.66, 395.0, 0.0],
 [8.35, 399.0, 0.0],
 [10.655999999999999, 400.0, 0.0],
 [25.92, 752.0, 0.0],
 [2.5940000000000003, 756.0, 0.0],
 [8.35, 760.0, 0.0],
 [12.488, 897.0, 0.0],
 [14.282, 902.0, 0.0],
 [0.12836, 905.0, 0.0],
 [15.38, 911.0, 0.0],
 [8.094, 916.0, 0.0],
 [12.0, 919.0, 0.0],
 [0.1049, 925.0, 0.0],
 [28.56, 930.0, 0.0],
 [44.18, 937.0, 0.0],
 [45.9, 943.0, 0.0],
 [34.38, 948.0, 0.0],
 [41.6, 969.0, 0.0],
 [39.46, 1006.0, 0.0],
 [65.24, 1010.0, 0.0],
 [48.92, 1015.0, 0.0],
 [26.22, 1022.0, 0.0],
 [36.76, 1026.0, 0.0],
 [0.2304, 1030.0, 0.0],
 [0.1112, 1035.0, 0.0],
 [40.08, 1038.0, 0.0],
 [8.984, 1042.0, 0.0],
 [0.06276, 1047.0, 0.0],
 [45.22, 1050.0, 0.0],
 [25.68, 1054.0, 0.0],
 [2.96, 1059.0, 0.0],
 [22.66, 1062.0, 0.0],
 [0.3418, 1066.0, 0.0],
 [0.28500000000000003, 1071.0, 0.0],
 [63.96, 1075.0, 0.0],
 [0.11444000000000001, 1079.0, 0.0],
 [63.85999999999999,

In [18]:
data='ecg'
filename='chfdb_chf13_45590.pkl'
model_name='LSTM'
augment=True
emsize=32
nhid=32         
nlayers=2
res_connection='store_true'
lr=0.0002
weight_decay=1e-4
clip=10
epochs=400
batch_size=64
eval_batch_size=64
bptt=50
teacher_forcing_ratio=0.7
dropout=0.2
tied='store_true'
seed=1111
device='cuda'
log_interval=10
save_interval=10
save_fig='store_true'
resume=False
pretrained=False
prediction_window_size=10

In [19]:
feature_dim = 15

In [20]:
model = model.RNNPredictor(rnn_type = model_name,
                           enc_inp_size=feature_dim,
                           rnn_inp_size = emsize,
                           rnn_hid_size = nhid,
                           dec_out_size=feature_dim,
                           nlayers = nlayers,
                           dropout = dropout,
                           tie_weights= tied,
                           res_connection=res_connection).to(device)
optimizer = optim.Adam(model.parameters(), lr= lr,weight_decay=weight_decay)
criterion = nn.MSELoss()

In [24]:
###############################################################################
def get_batch(args,source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len] # [ seq_len * batch_size * feature_size ]
    target = source[i+1:i+1+seq_len] # [ (seq_len x batch_size x feature_size) ]
    return data, target

def generate_output(args,epoch, model, gen_dataset, disp_uncertainty=True,startPoint=500, endPoint=3500):
    if save_fig:
        # Turn on evaluation mode which disables dropout.
        model.eval()
        hidden = model.init_hidden(1)
        outSeq = []
        upperlim95 = []
        lowerlim95 = []
        with torch.no_grad():
            for i in range(endPoint):
                if i>=startPoint:
                    # if disp_uncertainty and epoch > 40:
                    #     outs = []
                    #     model.train()
                    #     for i in range(20):
                    #         out_, hidden_ = model.forward(out+0.01*Variable(torch.randn(out.size())).cuda(),hidden,noise=True)
                    #         outs.append(out_)
                    #     model.eval()
                    #     outs = torch.cat(outs,dim=0)
                    #     out_mean = torch.mean(outs,dim=0) # [bsz * feature_dim]
                    #     out_std = torch.std(outs,dim=0) # [bsz * feature_dim]
                    #     upperlim95.append(out_mean + 2.58*out_std/np.sqrt(20))
                    #     lowerlim95.append(out_mean - 2.58*out_std/np.sqrt(20))

                    out, hidden = model.forward(out, hidden)

                    #print(out_mean,out)

                else:
                    out, hidden = model.forward(gen_dataset[i].unsqueeze(0), hidden)
                outSeq.append(out.data.cpu()[0][0].unsqueeze(0))


        outSeq = torch.cat(outSeq,dim=0) # [seqLength * feature_dim]

        target= preprocess_data.reconstruct(gen_dataset.cpu(), TimeseriesData.mean, TimeseriesData.std)
        outSeq = preprocess_data.reconstruct(outSeq, TimeseriesData.mean, TimeseriesData.std)
        # if epoch>40:
        #     upperlim95 = torch.cat(upperlim95, dim=0)
        #     lowerlim95 = torch.cat(lowerlim95, dim=0)
        #     upperlim95 = preprocess_data.reconstruct(upperlim95.data.cpu().numpy(),TimeseriesData.mean,TimeseriesData.std)
        #     lowerlim95 = preprocess_data.reconstruct(lowerlim95.data.cpu().numpy(),TimeseriesData.mean,TimeseriesData.std)

        plt.figure(figsize=(15,5))
        for i in range(target.size(-1)):
            plt.plot(target[:,:,i].numpy(), label='Target'+str(i),
                     color='black', marker='.', linestyle='--', markersize=1, linewidth=0.5)
            plt.plot(range(startPoint), outSeq[:startPoint,i].numpy(), label='1-step predictions for target'+str(i),
                     color='green', marker='.', linestyle='--', markersize=1.5, linewidth=1)
            # if epoch>40:
            #     plt.plot(range(startPoint, endPoint), upperlim95[:,i].numpy(), label='upperlim'+str(i),
            #              color='skyblue', marker='.', linestyle='--', markersize=1.5, linewidth=1)
            #     plt.plot(range(startPoint, endPoint), lowerlim95[:,i].numpy(), label='lowerlim'+str(i),
            #              color='skyblue', marker='.', linestyle='--', markersize=1.5, linewidth=1)
            plt.plot(range(startPoint, endPoint), outSeq[startPoint:,i].numpy(), label='Recursive predictions for target'+str(i),
                     color='blue', marker='.', linestyle='--', markersize=1.5, linewidth=1)

        plt.xlim([startPoint-500, endPoint])
        plt.xlabel('Index',fontsize=15)
        plt.ylabel('Value',fontsize=15)
        plt.title('Time-series Prediction on ' + data + ' Dataset', fontsize=18, fontweight='bold')
        plt.legend()
        plt.tight_layout()
        plt.text(startPoint-500+10, target.min(), 'Epoch: '+str(epoch),fontsize=15)
        save_dir = Path('result',data,filename).with_suffix('').joinpath('fig_prediction')
        save_dir.mkdir(parents=True,exist_ok=True)
        plt.savefig(save_dir.joinpath('fig_epoch'+str(epoch)).with_suffix('.png'))
        #plt.show()
        plt.close()
        return outSeq

    else:
        pass



def evaluate_1step_pred(args, model, test_dataset):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0
    with torch.no_grad():
        hidden = model.init_hidden(eval_batch_size)
        for nbatch, i in enumerate(range(0, test_dataset.size(0) - 1, bptt)):

            inputSeq, targetSeq = get_batch(args,test_dataset, i)
            outSeq, hidden = model.forward(inputSeq, hidden)

            loss = criterion(outSeq.view(batch_size,-1), targetSeq.view(batch_size,-1))
            hidden = model.repackage_hidden(hidden)
            total_loss+= loss.item()

    return total_loss / nbatch

def train(model, train_dataset,epoch):

    with torch.enable_grad():
        # Turn on training mode which enables dropout.
        model.train()
        total_loss = 0
        start_time = time.time()
        hidden = model.init_hidden(batch_size)
        for batch, i in enumerate(range(0, train_dataset.size(0) - 1, bptt)):
            inputSeq, targetSeq = get_batch(args,train_dataset, i)
            # inputSeq: [ seq_len * batch_size * feature_size ]
            # targetSeq: [ seq_len * batch_size * feature_size ]

            # Starting each batch, we detach the hidden state from how it was previously produced.
            # If we didn't, the model would try backpropagating all the way to start of the dataset.
            hidden = model.repackage_hidden(hidden)
            hidden_ = model.repackage_hidden(hidden)
            optimizer.zero_grad()

            '''Loss1: Free running loss'''
            outVal = inputSeq[0].unsqueeze(0)
            outVals=[]
            hids1 = []
            for i in range(inputSeq.size(0)):
                outVal, hidden_, hid = model.forward(outVal, hidden_,return_hiddens=True)
                outVals.append(outVal)
                hids1.append(hid)
            outSeq1 = torch.cat(outVals,dim=0)
            hids1 = torch.cat(hids1,dim=0)
            loss1 = criterion(outSeq1.view(batch_size,-1), targetSeq.view(batch_size,-1))

            '''Loss2: Teacher forcing loss'''
            outSeq2, hidden, hids2 = model.forward(inputSeq, hidden, return_hiddens=True)
            loss2 = criterion(outSeq2.view(batch_size, -1), targetSeq.view(batch_size, -1))

            '''Loss3: Simplified Professor forcing loss'''
            loss3 = criterion(hids1.view(batch_size,-1), hids2.view(batch_size,-1).detach())

            '''Total loss = Loss1+Loss2+Loss3'''
            loss = loss1+loss2+loss3
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()

            total_loss += loss.item()

            if batch % log_interval == 0 and batch > 0:
                cur_loss = total_loss / log_interval
                elapsed = time.time() - start_time
                print('| epoch {:3d} | {:5d}/{:5d} batches | ms/batch {:5.4f} | '
                      'loss {:5.2f} '.format(
                    epoch, batch, len(train_dataset) // bptt,
                                  elapsed * 1000 / log_interval, cur_loss))
                total_loss = 0
                start_time = time.time()

def evaluate(args, model, test_dataset):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    with torch.no_grad():
        total_loss = 0
        hidden = model.init_hidden(eval_batch_size)
        nbatch = 1
        for nbatch, i in enumerate(range(0, test_dataset.size(0) - 1, bptt)):
            inputSeq, targetSeq = get_batch(args,test_dataset, i)
            # inputSeq: [ seq_len * batch_size * feature_size ]
            # targetSeq: [ seq_len * batch_size * feature_size ]
            hidden_ = model.repackage_hidden(hidden)
            '''Loss1: Free running loss'''
            outVal = inputSeq[0].unsqueeze(0)
            outVals=[]
            hids1 = []
            for i in range(inputSeq.size(0)):
                outVal, hidden_, hid = model.forward(outVal, hidden_,return_hiddens=True)
                outVals.append(outVal)
                hids1.append(hid)
            outSeq1 = torch.cat(outVals,dim=0)
            hids1 = torch.cat(hids1,dim=0)
            loss1 = criterion(outSeq1.view(batch_size,-1), targetSeq.view(batch_size,-1))

            '''Loss2: Teacher forcing loss'''
            outSeq2, hidden, hids2 = model.forward(inputSeq, hidden, return_hiddens=True)
            loss2 = criterion(outSeq2.view(batch_size, -1), targetSeq.view(batch_size, -1))

            '''Loss3: Simplified Professor forcing loss'''
            loss3 = criterion(hids1.view(batch_size,-1), hids2.view(batch_size,-1).detach())

            '''Total loss = Loss1+Loss2+Loss3'''
            loss = loss1+loss2+loss3

            total_loss += loss.item()

    return total_loss / (nbatch+1)



In [25]:
# Loop over epochs.
if resume or pretrained:
    print("=> loading checkpoint ")
    checkpoint = torch.load(Path('save', data, 'checkpoint', filename).with_suffix('.pth'))
    args, start_epoch, best_val_loss = model.load_checkpoint(args,checkpoint,feature_dim)
    optimizer.load_state_dict((checkpoint['optimizer']))
    del checkpoint
    epoch = start_epoch
    print("=> loaded checkpoint")
else:
    epoch = 100
    start_epoch = 1
    best_val_loss = 0
    print("=> Start training from scratch")


if not pretrained:
    # At any point you can hit Ctrl + C to break out of training early.
    try:
        for epoch in range(start_epoch, epochs+1):

            epoch_start_time = time.time()
            train(model,train_dataset,epoch)
            val_loss = evaluate(args,model,test_dataset)
            print('-' * 89)
            print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.4f} | '.format(epoch, (time.time() - epoch_start_time),                                                                                        val_loss))
            print('-' * 89)

            generate_output(args,epoch,model,gen_dataset,startPoint=1500)

            if epoch%save_interval==0:
                # Save the model if the validation loss is the best we've seen so far.
                is_best = val_loss > best_val_loss
                best_val_loss = max(val_loss, best_val_loss)
                model_dictionary = {'epoch': epoch,
                                    'best_loss': best_val_loss,
                                    'state_dict': model.state_dict(),
                                    'optimizer': optimizer.state_dict(),
                                    'args':args
                                    }
                model.save_checkpoint(model_dictionary, is_best)

    except KeyboardInterrupt:
        print('-' * 89)
        print('Exiting from training early')


# Calculate mean and covariance for each channel's prediction errors, and save them with the trained model
print('=> calculating mean and covariance')
means, covs = list(),list()
train_dataset = TimeseriesData.batchify(args, TimeseriesData.trainData, bsz=1)
for channel_idx in range(model.enc_input_size):
    mean, cov = fit_norm_distribution_param(args,model,train_dataset[:TimeseriesData.length],channel_idx)
    means.append(mean), covs.append(cov)
model_dictionary = {'epoch': max(epoch,start_epoch),
                    'best_loss': best_val_loss,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'args': args,
                    'means': means,
                    'covs': covs
                    }
model.save_checkpoint(model_dictionary, True)
print('-' * 89)

=> Start training from scratch


TypeError: 'numpy.int64' object is not callable

In [27]:
train_dataset.size

150000

In [3]:
nyc_taxi_raw_path = 'nyc_taxi.csv'
labeled_data = []
with open(str(nyc_taxi_raw_path),'r') as f:
    for i, line in enumerate(f):
        tokens = [float(token) for token in line.strip().split(',')[1:]]
        tokens.append(1) if 150 < i < 250 or   \
                            5970 < i < 6050 or \
                            8500 < i < 8650 or \
                            8750 < i < 8890 or \
                            10000 < i < 10200 or \
                            14700 < i < 14800 \
                          else tokens.append(0)
        labeled_data.append(tokens)

In [19]:
labeled_data[0:100]

[[10844.0, 0.0, 1.0, 0],
 [8127.0, 30.0, 1.0, 0],
 [6210.0, 60.0, 1.0, 0],
 [4656.0, 90.0, 1.0, 0],
 [3820.0, 120.0, 1.0, 0],
 [2873.0, 150.0, 1.0, 0],
 [2369.0, 180.0, 1.0, 0],
 [2064.0, 210.0, 1.0, 0],
 [2221.0, 240.0, 1.0, 0],
 [2158.0, 270.0, 1.0, 0],
 [2515.0, 300.0, 1.0, 0],
 [4364.0, 330.0, 1.0, 0],
 [6526.0, 360.0, 1.0, 0],
 [11039.0, 390.0, 1.0, 0],
 [13857.0, 420.0, 1.0, 0],
 [15865.0, 450.0, 1.0, 0],
 [17920.0, 480.0, 1.0, 0],
 [20346.0, 510.0, 1.0, 0],
 [19539.0, 540.0, 1.0, 0],
 [20107.0, 570.0, 1.0, 0],
 [18984.0, 600.0, 1.0, 0],
 [17720.0, 630.0, 1.0, 0],
 [17249.0, 660.0, 1.0, 0],
 [18463.0, 690.0, 1.0, 0],
 [18908.0, 720.0, 1.0, 0],
 [18886.0, 750.0, 1.0, 0],
 [18178.0, 780.0, 1.0, 0],
 [19459.0, 810.0, 1.0, 0],
 [19546.0, 840.0, 1.0, 0],
 [20591.0, 870.0, 1.0, 0],
 [19380.0, 900.0, 1.0, 0],
 [18544.0, 930.0, 1.0, 0],
 [16228.0, 960.0, 1.0, 0],
 [15013.0, 990.0, 1.0, 0],
 [17203.0, 1020.0, 1.0, 0],
 [19525.0, 1050.0, 1.0, 0],
 [22966.0, 1080.0, 1.0, 0],
 [27598.0, 1110

In [18]:
pd.read_csv('../nyc_taxi.csv').columns

Index(['2014-07-01 00:00:00', '10844', '0', '1'], dtype='object')